This notebook goes through the following plan:
- preprocessing the data for topic modeling
- extract topics
- visualize topic importance by neighborhood

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
from matplotlib import pyplot as plt

listing_filename = 'listings.csv'
cols = ['id', 'name', 'neighbourhood_cleansed', 'neighborhood_overview', 'latitude', 'longitude']
df = pd.read_csv(listing_filename, usecols=cols)

# drop missing values
df.dropna(inplace=True)

## Prepare sentences composing each neighborhood
    - split to sentences
    - tokenize
    - lemmatize
    - discard stopwords and others

In [2]:
# Import spaCy and load the language library
import spacy
nlp = spacy.load('en_core_web_sm')
sentencizer = nlp.create_pipe("sentencizer")
nlp.add_pipe(sentencizer, before="parser")
doc = nlp("This is a sentence. This is another sentence.")

In [3]:
# Let's first create a function that decides what to exclude of the tokens of the sentence
def is_excluded(token, avoidwords=[]):
    '''
    This function decides whether to include 'token' to the list of lemmas
    INPUT
        token - token object in a doc
        avoidwords - additional words to exclude
    OUTPUT
        is_excluded - boolean - whether to exlude the token to the list of lemmas
    '''
    rules = [token.is_stop,
             token.is_punct,
             token.is_space,
             token.like_num,
             token.lemma_ in avoidwords
            ]
    return any(rules)

In [4]:
# Create a function splits a neighborhood_overview text into sentences, 
# discards some tokens and returns the sentences and the sentences after processing
def prepare_sentences(overview, avoidwords=[]):
    '''
    This function splits a neighborhood_overview text into sentences, 
    discards some tokens and returns the sentences and the sentences after processing
    INPUT
        overview - text from neighborhood_overview
        avoidwords - specific words to exclude
    OUTPUT
        sentences_lemmas - list of sentences of the overview text after processing
        sentences - boolean - list of sentences of the overview text
    '''
    if pd.notnull(overview): # or isinstance(text, str):
        doc = nlp(overview)
        sentences = []
        sentences_lemmas = []
        for sent in doc.sents:
            lemmas = []
            for token in sent:        
                if not is_excluded(token, avoidwords):
                    lemmas.append(token.lemma_)
                sent_lemmas = ' '.join(lemmas)
            if len(sent_lemmas):
                sentences_lemmas.append(sent_lemmas)
                sentences.append(sent.text)
        return sentences_lemmas, sentences
    else:
        print("warning: empty text")
        return [], []

# test
prepare_sentences(u"""Tesla is looking at buying a U.S. startup in Boston for $6 million.
                  Startups are becoming juicy minutes after it.""",
                 avoidwords=['Boston'])

(['Tesla look buy U.S. startup $', 'startup juicy minute'],
 ['Tesla is looking at buying a U.S. startup in Boston for $6 million.',
  '\n                  Startups are becoming juicy minutes after it.'])

In [5]:
# Apply prepare_sentences() to the whole dataset
# Each sentence can be linked to the neighborhood it is about using its index in the list
def prepare_data(dt, avoidwords=[]):
    '''
    This function applies prepare_sentences() to a dataframe
    INPUT
        dt - dataframe including neighbourhood_cleansed and neighborhood_overview
        avoidwords - specific words to exclude
    OUTPUT
        sents_lemmas - list of sentences from the neighborhood overviews after processing
        sents - list of sentences from the neighborhood overviews
        neighborhood_sentids - dictionary neighborhood --> ids of sentences in sents or sents_lemmas to
                               link each sentence in sents (or sents_lemmas) to the neighborhood it is about
    '''
    sents = []
    sents_lemmas = []
    #ntexts = []
    neighborhood_sentids = defaultdict(list)
    last_sentid = 0
    for index, overview, neighbourhood in zip(dt.index, dt.neighborhood_overview, dt.neighbourhood_cleansed):
        sent_lemmas, sent = prepare_sentences(overview, avoidwords)
        sents_lemmas += sent_lemmas
        sents += sent
        neighborhood_sentids[neighbourhood] += [i for i in range(last_sentid, last_sentid+len(sent))]
        last_sentid += len(sent)
    return sents_lemmas, sents, neighborhood_sentids

# test
sents_lemmas, sents, neighborhood_sentids = prepare_data(df.loc[:1, ["neighborhood_overview", 
                                                                     "neighbourhood_cleansed"]])
print("#### sents_lemmas:", sents_lemmas, sep='\n')
print("#### sents:", sents, sep='\n')
print("#### neighborhood_sentids:", neighborhood_sentids, sep='\n')

#### sents_lemmas:
['roslindale quiet convenient friendly', 'southern food try Redd Rozzie', 'Italian Delfino Sophia Grotto great', 'Birch St Bistro nice atmostphere little pricier', 'cook Fish market fresh fish daily Tony make sausage italian food wide variety delicious cheese chocolate Cheese Cellar Birch St.', 'room Roslindale diverse primarily residential neighborhood Boston', 'connect public transportation neighborhood easy access car', 'Roslindale Square nice business district supermarket', 'bank bakery etc', 'Guidebook recommendation', 'Arnold Arboretum step away']
#### sents:
['Roslindale is quiet, convenient and friendly.', " For Southern food try Redd's in Rozzie.", " Italian Delfino's or Sophia's Grotto are great.", 'Birch St Bistro has nice atmostphere--a little pricier.', "  If you are cooking the Fish Market has fresh fish daily; Tony's makes his own sausages and has Italian foods;  for  a wide variety of delicious cheeses and chocolates go to the Cheese Cellar on Birch S

In [6]:
neighborhoods = df["neighbourhood_cleansed"].unique()
X = df[["neighborhood_overview", "neighbourhood_cleansed"]]
# create sentences
# N.B. neighborhood names are excluded for the topics to remain unrelated to the neighborhoods 
sents_lemmas, sents, neighborhood_sentids = prepare_data(X, 
                        avoidwords=['Boston','neighborhood','jp', 'JP'] +' '.join(neighborhoods).split())

## Topic modeling
**What the topics are on the neighborhood overviews?**

Use Latent Dirichlet Allocation (LDA) to extract topics

In [7]:
# First create a document-term matrix 
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
dtm = cv.fit_transform(sents_lemmas)

In [8]:
from sklearn.decomposition import LatentDirichletAllocation
nbtopics = 5
LDA = LatentDirichletAllocation(n_components=nbtopics, random_state=42,
                               max_iter=80, evaluate_every=2)
# This can take awhile, we're dealing with a large amount of documents!
LDA.fit(dtm)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=2, learning_decay=0.7, learning_method='batch',
             learning_offset=10.0, max_doc_update_iter=100, max_iter=80,
             mean_change_tol=0.001, n_components=5, n_jobs=None,
             n_topics=None, perp_tol=0.1, random_state=42,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [9]:
LDA.n_iter_, LDA.perplexity(dtm)

(43, 709.9403504506598)

In [10]:
# Showing Top Words Per Topic:

for index,topic in enumerate(LDA.components_):
    print(f'THE TOP 20 WORDS FOR TOPIC #{index+1}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-20:]][::-1])
    print('\n')

THE TOP 20 WORDS FOR TOPIC #1
['walk', 'minute', 'downtown', 'close', 'access', 'station', 'easy', 'line', 'city', 'location', 'right', 'away', 'min', 'area', 'stop', 'locate', 'short', 'distance', 'bus', 'public']


THE TOP 20 WORDS FOR TOPIC #2
['restaurant', 'shop', 'walk', 'store', 'bar', 'great', 'street', 'cafe', 'food', 'away', 'block', 'good', 'minute', 'grocery', 'distance', 'coffee', 'italian', 'corner', 'local', 'market']


THE TOP 20 WORDS FOR TOPIC #3
['quiet', 'city', 'diverse', 'area', 'safe', 'family', 'live', 'residential', 'young', 'community', 'great', 'friendly', 'professional', 'people', 'home', 'love', 'student', 'street', 'locate', 'old']


THE TOP 20 WORDS FOR TOPIC #4
['mi', 'mile', 'center', 'museum', 'new', 'hall', 'restaurant', 'university', 'house', 'boston', 'fine', 'england', 'faneuil', 'trail', 'major', 'arts', 'freedom', 'home', 'garden', 'prudential']


THE TOP 20 WORDS FOR TOPIC #5
['street', 'walk', 'park', 'historic', 'beautiful', 'charles', 'river'

In [11]:
# For each neighborhood: How much proportion of the overviews are about topic X ?

topic_results = LDA.transform(dtm)
topic_asgn = topic_results.argmax(axis=1)

d = {**{"neighborhood": list(neighborhood_sentids.keys())}, **{f"topic {i+1}":[] for i in range(nbtopics)}}
d["sentences_count"] = []
d["dominant_topic"] = []
for nghbd, sentids in neighborhood_sentids.items():
    c = np.average(topic_results[sentids], axis=0)
    for i in range(nbtopics):
        d[f"topic {i+1}"].append(c[i])
    d["sentences_count"].append(len(sentids))
    d["dominant_topic"].append(np.argmax(c))
H = pd.DataFrame(d).sort_values(by="sentences_count", ascending=False).round(3)
H.set_index("neighborhood", inplace=True)
H.style.background_gradient(axis=0, subset=list(f"topic {i+1}" for i in range(nbtopics)))

,topic 1,topic 2,topic 3,topic 4,topic 5,sentences_count,dominant_topic
neighborhood,,,,,,,
Jamaica Plain,0.184,0.258,0.257,0.07,0.232,1018,1
South End,0.18,0.278,0.198,0.105,0.24,691,1
Dorchester,0.206,0.213,0.313,0.116,0.152,577,2
Back Bay,0.209,0.162,0.139,0.098,0.391,509,4
Allston,0.27,0.285,0.236,0.061,0.147,484,1
Fenway,0.279,0.175,0.147,0.216,0.183,436,0
South Boston,0.319,0.234,0.182,0.134,0.131,435,0
Beacon Hill,0.193,0.256,0.142,0.092,0.317,433,4
East Boston,0.253,0.28,0.277,0.067,0.123,347,1


## Visualize topic importance

In [13]:
# Visualize a specific topic importance with a map of 
# listings colored according to the topic proportion in their neighborhood

import plotly.express as px
# Specify the topic you you want to visualize
topic_col = "topic 2"
df[topic_col] = df["neighbourhood_cleansed"].apply(lambda n: H.loc[n, topic_col])
fig = px.scatter_mapbox(df, lat="latitude", lon="longitude", color=topic_col, 
                        hover_name="neighbourhood_cleansed", zoom=11,
                        center={'lat':42.32,'lon':-71.08}, width=700, height=700,
                        )
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## Remark: Apparently, results of Plotly cannot be displayed on github for being "dynamic"